# Zillow Clustering Project Scratchpad

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import os
from env import host, user, password
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import acquire
import prepare
import warnings
warnings.filterwarnings('ignore')

In [5]:
# connect to database, perform SQL query, write to csv, and convert to pandas DataFrame
df = acquire.get_zillow_cluster()

In [6]:
# what are we starting with?
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52438 entries, 0 to 52437
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52438 non-null  float64
 3   heatingorsystemtypeid         33934 non-null  float64
 4   buildingclasstypeid           0 non-null      float64
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13638 non-null  float64
 7   parcelid                      52438 non-null  int64  
 8   id                            52438 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52438 non-null  float64
 11  bedroomcnt                    52438 non-null  float64
 12  buildingqualitytypeid         33739 non-null  float64
 13  c

In [7]:
# rename parcelid as parcel_id and set as index

df = prepare.new_index(df)
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
parcel_id,,,,,,,,,,,,,,,,,,,,,
14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
12095076,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,3.0,...,6.037461e+13,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [8]:
# uses my nulls_by_col function and drops all columns with 50% or more null values
# uses my nulls_by_row function and drops all rows with 75% or more null values
df = prepare.handle_missing_values(df)

In [9]:
# creates dummy columns for each county
df = prepare.get_counties(df)

In [10]:
# creates my desired new feature columns
df = prepare.create_features(df)

In [11]:
# now what do we have?
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52359 entries, 14297519 to 12826780
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         52359 non-null  float64
 1   heatingorsystemtypeid         33934 non-null  float64
 2   id                            52359 non-null  int64  
 3   bathroomcnt                   52359 non-null  float64
 4   bedroomcnt                    52359 non-null  float64
 5   buildingqualitytypeid         33738 non-null  float64
 6   calculatedbathnbr             52297 non-null  float64
 7   calculatedfinishedsquarefeet  52351 non-null  float64
 8   finishedsquarefeet12          52187 non-null  float64
 9   fips                          52359 non-null  float64
 10  fullbathcnt                   52297 non-null  float64
 11  latitude                      52359 non-null  float64
 12  longitude                     52359 non-null  floa

In [13]:
# columns to explore possibility of either dropping or imputing...
col_explore = ['heatingorsystemtypeid', 'buildingqualitytypeid', 'propertyzoningdesc', 'unitcnt', 'heatingorsystemdesc']
for col in col_explore:
    print(col)
    print(df[col].value_counts())
    print('')

heatingorsystemtypeid
2.0     20736
7.0     12564
6.0       517
20.0       85
13.0       16
1.0         7
18.0        6
10.0        2
24.0        1
Name: heatingorsystemtypeid, dtype: int64

buildingqualitytypeid
6.0     10289
8.0      8244
4.0      8154
7.0      3427
5.0      1513
9.0      1121
11.0      514
10.0      230
3.0       152
12.0       81
1.0         8
2.0         5
Name: buildingqualitytypeid, dtype: int64

propertyzoningdesc
LAR1          6495
LARS          1385
LBR1N         1111
SCUR2          712
LARE11         612
              ... 
LR7OOO*          1
LARU             1
LRCR*            1
AHRPD51U-R       1
COML             1
Name: propertyzoningdesc, Length: 1286, dtype: int64

unitcnt
1.0    33816
2.0       29
3.0        1
Name: unitcnt, dtype: int64

heatingorsystemdesc
Central       20736
Floor/Wall    12564
Forced air      517
Solar            85
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsys